# Welcome!
Below, we will learn to implement and train a policy to play atari-pong, using only the pixels as input. We will use convolutional neural nets, multiprocessing, and pytorch to implement and train our policy. Let's get started!

In [1]:
# install package for displaying animation
!pip install JSAnimation

# custom utilies for displaying animation, collecting rollouts and more
import pong_utils

%matplotlib inline

# check which device is being used. 
# I recommend disabling gpu until you've made sure that the code runs
device = pong_utils.device
print("using device: ",device)

  Running setup.py bdist_wheel for JSAnimation ... done
  Stored in directory: /root/.cache/pip/wheels/3c/c2/b2/b444dffc3eed9c78139288d301c4009a42c0dd061d3b62cead
Successfully built JSAnimation
using device:  cpu


In [3]:
# render ai gym environment
import gym
import time

# PongDeterministic does not contain random frameskip
# so is faster to train than the vanilla Pong-v4 environment
env = gym.make('Pendulum-v0')

env.seed(2)
print('observation space:', env.observation_space)
print('action space:', env.action_space)


# we will only use the actions 'RIGHTFIRE' = 4 and 'LEFTFIRE" = 5
# the 'FIRE' part ensures that the game starts again after losing a life
# the actions are hard-coded in pong_utils.py

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
observation space: Box(3,)
action space: Box(1,)


# Preprocessing
To speed up training, we can simplify the input by cropping the images and use every other pixel



# Policy

## Exercise 1: Implement your policy
 
Here, we define our policy. The input is the stack of two different frames (which captures the movement), and the output is a number $P_{\rm right}$, the probability of moving left. Note that $P_{\rm left}= 1-P_{\rm right}$

In [201]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal


# set up a convolutional neural net
# the output is the probability of moving right
# P(left) = 1-P(right)
class Policy(nn.Module):
    
    def __init__(self, s_size=3, h_size=300, a_size=1):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h_size)
        
        self.mu = nn.Linear(h_size, a_size)
        self.std = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        
        mu = F.tanh(self.mu(x))
        std = F.softplus(self.std(x))
        return mu, std

    def act(self, state, batch_mode=False):
        if not batch_mode:
            state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        mu, std = self.forward(state)
        
        m = Normal(mu, std)
        action = m.sample().clamp(-2,2) # obs action space is betwween -2 and 2
        
        if not batch_mode:
            action = action.item() 
        
        return action, m.cdf(action), m.entropy()
# run your own policy!
policy=Policy().to(device)
#policy=pong_utils.Policy().to(device)

# we use the adam optimizer with learning rate 2e-4
# optim.SGD is also possible
import torch.optim as optim
optimizer = optim.Adam(policy.parameters(), lr=1e-3)

In [202]:
policy

Policy(
  (fc1): Linear(in_features=3, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=300, bias=True)
  (mu): Linear(in_features=300, out_features=1, bias=True)
  (std): Linear(in_features=300, out_features=1, bias=True)
)

# Game visualization
pong_utils contain a play function given the environment and a policy. An optional preprocess function can be supplied. Here we define a function that plays a game and shows learning progress

# Function Definitions
Here you will define key functions for training. 

## Exercise 2: write your own function for training
(what I call scalar function is the same as policy_loss up to a negative sign)

### PPO
Later on, you'll implement the PPO algorithm as well, and the scalar function is given by
$\frac{1}{T}\sum^T_t \min\left\{R_{t}^{\rm future}\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)},R_{t}^{\rm future}{\rm clip}_{\epsilon}\!\left(\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)}\right)\right\}$

the ${\rm clip}_\epsilon$ function is implemented in pytorch as ```torch.clamp(ratio, 1-epsilon, 1+epsilon)```

In [34]:
states =  envs.reset()

In [35]:
states

array([[-0.07472608, -0.9972041 ,  0.4784946 ],
       [ 0.97676342, -0.21432038,  0.46877921],
       [ 0.06720443,  0.99773923,  0.09108689],
       [-0.99731068, -0.07328989,  0.77271367],
       [-0.32019129,  0.94735291,  0.33915817],
       [-0.88013181, -0.47472939,  0.44229691],
       [-0.94415689,  0.32949623,  0.03627558],
       [-0.76469761, -0.6443893 ,  0.49232976]])

In [111]:
test

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9]])

In [ ]:
# collect trajectories for a parallelized parallelEnv object
def collect_trajectories(envs, policy, tmax=200, nrand=5):
    
    # number of parallel instances
    n=len(envs.ps)

    #initialize returning lists and start the game!
    state_list=[]
    reward_list=[]
    prob_list=[]
    action_list=[]

    states = envs.reset()

    states = torch.from_numpy(states).float() 

    for t in range(tmax):
        
        actions, probs, entrops = policy.act(states, batch_mode=True)        
        states, rewards, dones, _  = envs.step(actions)
        states = torch.from_numpy(states).float()
        # store the result
        state_list.append(states)
        reward_list.append(rewards)
        prob_list.append(probs.squeeze().cpu().detach().numpy())
        action_list.append(actions.squeeze().cpu().detach().numpy())
        
        # stop if any of the trajectories is done
        # we want all the lists to be retangular
        if dones.any():
            break

    #state_list = torch.stack(state_list).squeeze()
    #prob_list = torch.stack(prob_list).squeeze()
    #action_list = torch.stack(action_list).squeeze()
    # return pi_theta, states, actions, rewards, probability
    return prob_list, state_list, \
        action_list, reward_list

In [ ]:
old_probs, states, actions, rewards = \
             collect_trajectories(envs, policy, tmax=300)

In [ ]:
old_probs.shape

In [ ]:
np.asarray(rewards)

In [ ]:
test = torch.from_numpy(np.arange(10).reshape((5,2)))
discount = 0.99**np.arange(len(test))

In [204]:
test, discount

(tensor([[ 0,  1],
         [ 2,  3],
         [ 4,  5],
         [ 6,  7],
         [ 8,  9]]),
 array([ 1.        ,  0.99      ,  0.9801    ,  0.970299  ,  0.96059601]))

In [ ]:
def clipped_surrogate(policy, old_probs, states, actions, rewards,
                      discount = 0.995, epsilon=0.1, beta=0.01):

    ########
    ## 
    ## WRITE YOUR OWN CODE HERE
    ##
    ########
    
    actions = torch.tensor(actions, dtype=torch.int8, device=device)

    # convert states to policy (or probability)
    new_probs = pong_utils.states_to_prob(policy, states)
    new_probs = torch.where(actions == pong_utils.RIGHT, new_probs, 1.0-new_probs)

    # include a regularization term
    # this steers new_policy towards 0.5
    # prevents policy to become exactly 0 or 1 helps exploration
    # add in 1.e-10 to avoid log(0) which gives nan
    entropy = -(new_probs*torch.log(old_probs+1.e-10)+ \
        (1.0-new_probs)*torch.log(1.0-old_probs+1.e-10))

    return torch.mean(beta*entropy)


In [205]:
def clipped_surrogate_continuous(policy, old_probs, states, actions, rewards,
                      discount=0.995,
                      epsilon=0.1, beta=0.01):

    discount = discount**np.arange(len(rewards))
    rewards = np.asarray(rewards)*discount[:,np.newaxis]
    
    # convert rewards to future rewards
    rewards_future = rewards[::-1].cumsum(axis=0)[::-1]
    
    mean = np.mean(rewards_future, axis=1)
    std = np.std(rewards_future, axis=1) + 1.0e-10

    rewards_normalized = (rewards_future - mean[:,np.newaxis])/std[:,np.newaxis]
    rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)
    
    actions = torch.tensor(actions, dtype=torch.int8, device=device)
    old_probs = torch.tensor(old_probs, dtype=torch.float, device=device)
    
    _, new_probs, _ = policy.act(torch.stack(states), batch_mode=True)  
    
    new_probs = new_probs.squeeze()
    # ratio for clipping

    ratio = new_probs/old_probs

    # clipped function
    clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)
    clipped_surrogate = torch.min(ratio*rewards, clip*rewards)

    # include a regularization term
    # this steers new_policy towards 0.5
    # add in 1.e-10 to avoid log(0) which gives nan
    entropy = -(new_probs*torch.log(old_probs+1.e-10)+ \
        (1.0-new_probs)*torch.log(1.0-old_probs+1.e-10))

    
    # this returns an average of all the entries of the tensor
    # effective computing L_sur^clip / T
    # averaged over time-step and number of trajectories
    # this is desirable because we have normalized our rewards
    return torch.mean(clipped_surrogate + beta*entropy)


# Training
We are now ready to train our policy!
WARNING: make sure to turn on GPU, which also enables multicore processing. It may take up to 45 minutes even with GPU enabled, otherwise it will take much longer!

In [ ]:
from parallelEnv import parallelEnv

import numpy as np
# keep track of how long training takes
# WARNING: running through all 800 episodes will take 30-45 minutes

optimizer = optim.Adam(policy.parameters(), lr=1e-3)
# training loop max iterations
episode = 2000

# widget bar to display progress
!pip install progressbar
import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()


envs = parallelEnv('Pendulum-v0', n=8, seed=1234)

discount_rate = .99
epsilon = 0.1
beta = .01
tmax = 320
SGD_epoch = 4

# keep track of progress
mean_rewards = []

for e in range(episode):

    # collect trajectories
    old_probs, states, actions, rewards = \
        collect_trajectories(envs, policy, tmax=tmax)
        
    total_rewards = np.sum(rewards, axis=0)


    # gradient ascent step
    for _ in range(SGD_epoch):
        
        # uncomment to utilize your own clipped function!
        # L = -clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=epsilon, beta=beta)

        #L = -pong_utils.clipped_surrogate(policy, old_probs, states, actions, rewards,
        #                                  epsilon=epsilon, beta=beta)
    
        L = -clipped_surrogate_continuous(policy, old_probs, states, actions, rewards,
                                          epsilon=epsilon, beta=beta)
        #print(L.grad_fn)
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=.999
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=.995
    
    # get the average reward of the parallel environments
    mean_rewards.append(np.mean(total_rewards))
    
    # display some progress every 20 iterations
    if (e+1)%20 ==0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(total_rewards)))
        print(total_rewards)
        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.

Episode: 20, score: -1231.045602
[ -971.20541383 -1583.80014121 -1069.67028034 -1581.71451825 -1526.56766599
 -1070.46168981  -982.14652612 -1062.79857795]


Episode: 40, score: -1253.604319
[-1289.46725265 -1380.48150182 -1190.1387016  -1584.51859791 -1011.45376654
 -1152.25220797 -1454.5734176   -965.94910449]


Episode: 60, score: -1238.770824
[ -975.46196931 -1552.01102686  -969.43518177 -1176.37503771 -1002.19351655
 -1383.68296222 -1254.22733577 -1596.77956141]


Episode: 80, score: -1155.663544
[-1429.67617695 -1596.63135004  -908.36048009 -1290.11951899 -1087.87324337
  -870.51307551 -1066.43260975  -995.70189952]


...

Episode: 640, score: -1285.443108
[-1349.52682538 -1388.22189097 -1500.3597072  -1360.83382211 -1219.1449741
  -931.09510624 -1051.62123469 -1482.74130698]


Episode: 660, score: -1316.742350
[-1537.66093553 -1610.68639936 -1088.44862568 -1296.64018297 -1578.9440308
 -1609.4468893   -952.543213    -859.56852297]


Episode: 680, score: -1327.250777
[-1054.97900696  -971.71153439 -1710.32022819 -1442.63633267 -1015.7318176
 -1461.32908427 -1533.74969202 -1427.54851879]


Episode: 700, score: -1209.919486
[-1562.10572067 -1374.75560369 -1070.96280406 -1230.03322017  -998.6241741
 -1085.40686246 -1485.27532305  -872.19218219]


Episode: 720, score: -1323.041588
[-1599.68745628 -1592.25375223 -1518.41612485  -761.64989352 -1007.92681588
 -1746.70241686 -1493.27984658  -864.41639392]


In [ ]:
pong_utils.play(env, policy, time=200) 

In [ ]:
# save your policy!
torch.save(policy, 'PPO.policy')

# load policy if needed
# policy = torch.load('PPO.policy')

# try and test out the solution 
# make sure GPU is enabled, otherwise loading will fail
# (the PPO verion can win more often than not)!
#
# policy_solution = torch.load('PPO_solution.policy')
# pong_utils.play(env, policy_solution, time=2000) 